In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Setup Functions

In [2]:
# valHTML is a Python response from a get.
def findResultsPageMax(valHTML):
    rtnVal = -1
    valBS = BeautifulSoup(valHTML.text,'lxml')
    valTags = valBS.find_all('li')
    tmpURLs = []
    for i in valTags:
        valAs = i.find('a')
        try:
            tmpURLs.append(valAs.attrs['href'])
        except:
            None
    tmpAllURLs = pd.DataFrame({'URLs': tmpURLs})
    tmpSubset = tmpAllURLs.loc[tmpAllURLs['URLs'].str.contains('/sbirsearch/award/all\?page=')].copy()
    tmpSubset['Number'] = tmpSubset['URLs'].str.extract('([0-9]+)$',expand=False)
    tmpSubset['Number'] = pd.to_numeric(tmpSubset['Number'])
    tmpSubset.dropna(inplace=True)
    rtnVal = int(max(tmpSubset['Number']))
    return(rtnVal)

In [3]:
def getResultPage(valURL,valMaxTries):
    valCnt = 1
    isBad = True
    rtnVal = None
    while(isBad and valCnt < valMaxTries):
        try:
            tmpResponse = requests.get(valURL)
            if(tmpResponse.status_code==200):
                isBad = False
                rtnVal = tmpResponse
        except:
            None
        valCnt += 1
    return(rtnVal)

In [4]:
def getResultSBIRLinks(valHTML):
    valBS = BeautifulSoup(valHTML.text,'lxml')
    valTags = valBS.find_all('h3')
    tmpURLs = []
    for i in valTags:
        valAs = i.find('a')
        tmpURLs.append(valAs.attrs['href'])
    return(tmpURLs)

# Get SBIR Results

In [5]:
%%time
valRetries = 10
valMin = 0
valURLBase = r'https://www.sbir.gov/sbirsearch/award/all?page='

valMax = findResultsPageMax(getResultPage(valURLBase+str(valMin),100))
valRange = range(valMin,20)#valMax+1)
tmpResultsPage = []
tmpSBIRLinks = pd.DataFrame()

for i in valRange:
    tmpURL = valURLBase+str(i)
    valSearchResultsPage = getResultPage(tmpURL,valRetries)
    tmpResultsPage.append(valSearchResultsPage)
    valSBIRLinks = getResultSBIRLinks(valSearchResultsPage)
    tmpDF = pd.DataFrame({'SBIRLink': valSBIRLinks})
    tmpDF['ResultsPage'] = i
    tmpSBIRLinks = tmpSBIRLinks.append(tmpDF)
    if((i % 1000)==0):
        print("Finished Retrieving "+str(i)+" Pages...")

Finished Retrieving 0 Pages...
CPU times: user 3.02 s, sys: 76 ms, total: 3.09 s
Wall time: 29.4 s


In [6]:
%%time
tmpSBIRLinks['SBIRLinkRev'] = r'https://www.sbir.gov'+tmpSBIRLinks['SBIRLink']
tmpSBIRLinks['SBIResponse'] = tmpSBIRLinks['SBIRLinkRev'].apply(lambda x: getResultPage(x,valRetries))
tmpSBIRLinks['SBIRText'] = tmpSBIRLinks['SBIResponse'].apply(lambda x: x.text)

CPU times: user 3.89 s, sys: 402 ms, total: 4.29 s
Wall time: 2min 33s


In [7]:
tmpSBIRLinks.to_excel('./SBIRAwards.xlsx')